<a href="https://colab.research.google.com/github/abigail7318/2023Vision/blob/main/yad2Name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apscheduler requests_html regex beautifulsoup4 python-bidi pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
def listToString(data_list):
  data_str= ''
  for datum in data_list:
    data_str+= str(datum) + ','
  return data_str[:-1]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import re
import asyncio
from bidi.algorithm import get_display
import datetime
import random
import pandas as pd
import os
import time
from apscheduler.schedulers.blocking import BlockingScheduler

def getRawProductDataFile(path='/content/product-raw-data'):
    f = open(path, 'r')
    return f.read()

def saveRawText(path,text):

    log_file = open(path, "a")
    log_file.write(text)
    log_file.close()

def getNotExistingProductData(path='/content/drive/MyDrive/yad2/city-and-phone'):

  df = pd.read_csv(path)
  text = getRawProductDataFile()
  print(text[0:100])
  if text:
    last_product_id = str(df['Product Id'].iloc[-1])
    print(f'Last Id: {last_product_id}')
    try:
      start_index = text.index(last_product_id)+1
      return text[start_index:-1]
    except:
      return text
  else:
    return text

def saveLogMessage(message):

      print('\n',message)
      saveRawText(path='/content/yad2_log.txt',text=f'\n{message}')

async def getHtml(url):

    try:
        scheduler = BlockingScheduler()

        asession = AsyncHTMLSession()

        response = await asession.get(url)

        if response:

              saveLogMessage('response is goood')

        else:

                saveLogMessage('response oh no')

        await response.html.arender(sleep=random.uniform(5.22222, 8.787556))

        html = response.html.raw_html

        if html:

              saveLogMessage('html is goood')
              print(html)
              return html
        else:

              saveLogMessage('html oh no')

        saveLogMessage(f'\nstatus_code:{response.status_code}')
    except Exception as e:

      saveLogMessage('cricessssssssssssss\n', e)


def byRegex(regex_pattern, text):

    result_list = []

    try:

        compiled_pattern = regex.compile(regex_pattern, regex.IGNORECASE)

        for match in regex.finditer(compiled_pattern, text):

            saveLogMessage(f'Found Sequence: {match.group(0)}')

            result_list.append(match.group(0))

        return result_list if result_list else ['Not Found']

    except:

        saveLogMessage('Regex not found')

        return ['Not Found']

async def getName(product_id):

  url = r'https://market.yad2.co.il/tools/proxy/seller/'
  try:
      asession = AsyncHTMLSession()
      r = await asession.get(url + product_id)
      if r:
            print('r is goood')
      else:
            print('r oh no')
      await r.html.arender(sleep=random.uniform(3.22222, 8.787556))
      resp=r.html.raw_html
      if resp:
          print('resp is goood')
      else:
          print('r oh no')
      print('\nstatus_code:', r.status_code)

      soup = BeautifulSoup(resp, 'html.parser')
      print(soup.get_text())
  except Exception as e:
      print('crisissssssssssssss\n', e)


async def getPersonalData(product_id):
     person_dict = {}
     person_dict.update({'Product Id': product_id, 'Name': await getName(product_id)})
     savePersonalData(person_dict,path ='/content/names')

def savePersonalData(person_dict,path='/content/drive/MyDrive/yad2/yad2-content'):

    try:
        is_header = False if  os.path.exists(path) else True
        df = pd.DataFrame([person_dict])
        df.to_csv(path_or_buf=path, header=is_header,mode='a',index=False)
        saveLogMessage('Human Info Saved')
    except:
        saveLogMessage('ERROR: Human Info npt Saved')

main_html = getNotExistingProductData()

for product_id in byRegex(r'(?<=data-product-id=")\d+[^"]',str(main_html)):
            await getPersonalData(product_id)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 1048576: invalid start byte

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Copy of yad2_product_ids.txt'

related link = https://api.fastsimon.com/related_products_suggest?src=JSSDK&product_id=7429412552764&UUID=a93cd17e-84be-4302-b068-8b2f37c1c415&store_id=59078606908&products=%5b%5d&specs=%5b%7B%22sources%22:%22similar_products_by_attributes%22,%22max_suggest%22:%2210%22,%22categories%22:%22%22,%22title%22:%22%22%7D%5d&cart_token=d3b106b3b39f556b2379bdb94a42013d&with_product_attributes=true

yad2 =